# Colab Setup  
> Make sure you configure notebook with GPU: Click Edit->notebook settings->hardware accelerator->GPU

> Uncomment the following cell after opening in Google colab. (Do not uncomment it in local setup.)  

<a target="_blank" href="https://colab.research.google.com/github/SEED-VT/FedDebug/blob/main/fault-localization/artifact.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



In [1]:
# !pip install pytorch-lightning
# !pip install diskcache
# !pip install dotmap
# !pip install torch torchvision torchaudio
# !pip install matplotlib
# !git clone https://github.com/SEED-VT/FedDebug.git
# # appending the path
# import sys
# sys.path.append("FedDebug/fault-localization/")

# Running Code

In [2]:
import logging
import matplotlib.pyplot as plt
import time
from dotmap import DotMap
from pytorch_lightning import seed_everything
from torch.nn.init import kaiming_uniform_ 
from utils.faulty_client_localization.FaultyClientLocalization import FaultyClientLocalization
from utils.faulty_client_localization.InferenceGuidedInputs import InferenceGuidedInputs
from utils.FLSimulation import trainFLMain
from utils.fl_datasets import initializeTrainAndValidationDataset
from utils.util import aggToUpdateGlobalModel
from utils.util import testAccModel



logging.basicConfig(filename='example.log', level=logging.ERROR)
logger = logging.getLogger("pytorch_lightning")
seed_everything(786)



def evaluateFaultLocalization(predicted_faulty_clients_on_each_input, true_faulty_clients):
    true_faulty_clients = set(true_faulty_clients)
    detection_acc = 0
    for pred_faulty_clients in predicted_faulty_clients_on_each_input:
        print(f"+++ Faulty Clients {pred_faulty_clients}")
        correct_localize_faults = len(
            true_faulty_clients.intersection(pred_faulty_clients))
        acc = (correct_localize_faults/len(true_faulty_clients))*100
        detection_acc += acc
    fault_localization_acc = detection_acc / \
        len(predicted_faulty_clients_on_each_input)
    return fault_localization_acc


def runFaultyClientLocalization(client2models, exp2info, num_bugs, random_generator=kaiming_uniform_, apply_transform=True, k_gen_inputs=10, na_threshold=0.003, use_gpu=True):
    print(">  Running FaultyClientLocalization ..")
    input_shape = list(exp2info['data_config']['single_input_shape'])
    generate_inputs = InferenceGuidedInputs(client2models, input_shape, randomGenerator=random_generator, apply_transform=apply_transform,
                                            dname=exp2info['data_config']['name'], min_nclients_same_pred=5, k_gen_inputs=k_gen_inputs)
    selected_inputs, input_gen_time = generate_inputs.getInputs()

    start = time.time()
    faultyclientlocalization = FaultyClientLocalization(
        client2models, selected_inputs, use_gpu=use_gpu)

    potential_benign_clients_for_each_input = faultyclientlocalization.runFaultLocalization(
        na_threshold, num_bugs=num_bugs)
    fault_localization_time = time.time()-start
    return potential_benign_clients_for_each_input, input_gen_time, fault_localization_time





# ====== Simulation ===== 

args = DotMap()
args.lr = 0.001
args.weight_decay = 0.0001
args.batch_size = 512
args.epochs = 10  # range 10-25
args.dataset = "cifar10" # ['cifar10', 'femnist']
args.clients = 30 # keep under 30 clients and use Resnet18, Resnet34, or Densenet to evaluate on Colab 
args.faulty_clients_ids = "0,1,2,3,4" # can be multiple clients separated by comma e.g. "0,1,2"  but keep under args.clients clients and at max less than 7 
args.sampling = "iid" # [iid, "niid"] 
args.noise_rate = 1  # noise rate 0 to 1 




/home/gulzar/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 786
Global seed set to 786


### Figure 10-a, Resnet and Cifar10

In [3]:
args.model = "resnet18" # [resnet18, resnet34, resnet50, densenet121, vgg16]
c2ms, exp2info = trainFLMain(args)
client2models = {k: v.model.eval() for k, v in c2ms.items()}


# Fault localazation
neuron_act_thresholds = [0, 0.001, 0.003, 0.005, 0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3, 0.5, 1]

na2acc = []

for na_t in neuron_act_thresholds:
    potential_faulty_clients, _, _ = runFaultyClientLocalization(
        client2models=client2models, exp2info=exp2info, num_bugs=len(exp2info['faulty_clients_ids']), na_threshold=na_t)

    fault_acc = evaluateFaultLocalization(
        potential_faulty_clients, exp2info['faulty_clients_ids'])


    print(f"#Figure 10: NA threshold: {na_t} Fault Localization Accuracy: {fault_acc}, Distribution: {args.sampling},  Faulty clients: {len(args.faulty_clients_ids.split(','))}, Total Clients: {args.clients}, Architecture: {args.model}, Dataset: {args.dataset}")
    na2acc.append(fault_acc)



x = neuron_act_thresholds 

fig, axes = plt.subplots(1, 1) 
axes.plot(x, na2acc, 'g--o') 
# axes[1].plot(x, y2_fault_acc, 'm--o')

axes.set_xlabel('Noise Rate')
axes.set_ylabel('Fault Localization Accuracy (%)')
axes.set_title("Paper Figure-10 (a) ResNet and CIFAR-10")


axes.set_ylim(0, 105)


fig.set_figwidth(5)
plt.tight_layout()




  ***Simulating FL setup iid_resnet18_cifar10_clients_30_faulty_[0, 1, 2, 3, 4]_bsize_512_epochs_10_lr_0.001 ***
Files already downloaded and verified
Files already downloaded and verified
Spliting Datasets 50000 into parts:[1686, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666]
input shape, torch.Size([1, 3, 32, 32])
Training : .storage/checkpoints/iid_resnet18_cifar10_clients_30_faulty_[0, 1, 2, 3, 4]_bsize_512_epochs_10_lr_0.001/faulty_client_0_noise_rate_1_classes.ckpt


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Train mod batch = 150, and drop_last = False


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 9: 100%|██████████| 4/4 [00:01<00:00,  3.54it/s, loss=2.31, train_acc=0.0714, train_loss=2.310, val_acc=0.101, val_loss=2.360]

...

Train mod batch = 130, and drop_last = False
Epoch 9: 100%|██████████| 4/4 [00:01<00:00,  3.68it/s, loss=2.32, train_acc=0.0705, train_loss=2.340, val_acc=0.0978, val_loss=2.530]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 4/4 [00:01<00:00,  3.67it/s, loss=2.32, train_acc=0.0705, train_loss=2.340, val_acc=0.0978, val_loss=2.530]
Training : .storage/checkpoints/iid_resnet18_cifar10_clients_30_faulty_[0, 1, 2, 3, 4]_bsize_512_epochs_10_lr_0.001/faulty_client_3_noise_rate_1_classes.ckpt


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Train mod batch = 130, and drop_last = False
Epoch 0:  75%|███████▌  | 3/4 [00:01<00:00,  2.80it/s, loss=2.56, train_acc=0.0869, train_loss=2.520, val_acc=-1.00, val_loss=100.0]

### Figure 10-C, Densenet121 and Cifar10

In [ ]:

args.model = "densenet121" # [resnet18, resnet34, resnet50, densenet121, vgg16]
c2ms, exp2info = trainFLMain(args)
client2models = {k: v.model.eval() for k, v in c2ms.items()}


# Fault localazation
neuron_act_thresholds = [0, 0.001, 0.003, 0.005, 0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3, 0.5, 1]

na2acc = []

for na_t in neuron_act_thresholds:
    potential_faulty_clients, _, _ = runFaultyClientLocalization(
        client2models=client2models, exp2info=exp2info, num_bugs=len(exp2info['faulty_clients_ids']), na_threshold=na_t)

    fault_acc = evaluateFaultLocalization(
        potential_faulty_clients, exp2info['faulty_clients_ids'])


    print(f"#Figure 10: NA threshold: {na_t} Fault Localization Accuracy: {fault_acc}, Distribution: {args.sampling},  Faulty clients: {len(args.faulty_clients_ids.split(','))}, Total Clients: {args.clients}, Architecture: {args.model}, Dataset: {args.dataset}")
    na2acc.append(fault_acc)



x = neuron_act_thresholds 

fig, axes = plt.subplots(1, 1) 
axes.plot(x, na2acc, 'm--o') 
# axes[1].plot(x, y2_fault_acc, 'm--o')

axes.set_xlabel('Noise Rate')
axes.set_ylabel('Fault Localization Accuracy (%)')
axes.set_title("Paper Figure-10 (c) DenseNet121 and CIFAR-10")


axes.set_ylim(0, 105)


fig.set_figwidth(5)
plt.tight_layout()


